In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
base_url

'https://api.deepseek.com'

In [ ]:
# Chat接口
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        { "role": "system", "content": "你是一个大模型专业的AI助手" },
        { "role": "user", "content": "请解释一下Transformer的核心机制" },
    ]
)
response

ChatCompletion(id='ea2c15ef-3362-488a-a4c8-a2eef75bba4c', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='好的，Transformer 的核心机制可以概括为：**一种完全基于自注意力机制和前馈神经网络，并行处理整个序列，以捕捉长距离依赖关系的编码器-解码器架构。**\n\n它的核心突破在于摒弃了传统的循环神经网络（RNN）和卷积神经网络（CNN），解决了序列建模中“长距离依赖”和“训练并行化”两大难题。\n\n下面我们来分解其最核心的几个机制：\n\n### 1. 自注意力机制\n这是 Transformer 的灵魂。它的核心思想是：**序列中的每个元素（如一个词）都应该与序列中的所有其他元素进行交互，从而动态地计算出一个能融合全局信息的“新表示”。**\n\n**运作过程（以“我爱人工智能”这句话为例）：**\n1.  **生成三个向量**：对于每个输入词（如“爱”），通过三个不同的权重矩阵，生成三个向量：\n    *   **查询向量**：代表“我在寻找什么”。\n    *   **键向量**：代表“我有什么可提供的”。\n    *   **值向量**：代表“我的实际内容是什么”。\n2.  **计算注意力分数**：用“爱”的 **Q** 去与序列中所有词（包括“爱”自己）的 **K** 进行点积，得到一组分数。分数越高，表示两个词在当前任务下的关联性越强。\n    *   例如，“爱”的 Q 与“我”的 K 点积可能得分高（谁在爱？），与“人工智能”的 K 点积得分也可能高（爱什么？）。\n3.  **归一化与加权求和**：将这些分数通过 Softmax 归一化为概率分布（注意力权重）。然后用这些权重对各个词的 **V** 进行加权求和，得到“爱”这个词最终的**上下文感知表示**。\n    *   这个新表示不再是一个孤立的“爱”字，而是包含了“**我**”和“**人工智能**”信息的“爱”。\n\n**公式表示（缩放点积注意力）：**\n\\[ \\text{Attention}(Q, K, V) = \\text{softmax}\\l

In [4]:
# 流式响应
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

stream = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        { "role": "system", "content": "你是一个大模型专业的AI助手" },
        { "role": "user", "content": "请分点简述注意力机制的核心思想" },
    ],
    stream=True,
)
for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="", flush=True)

注意力机制的核心思想是通过动态分配权重，使模型能够有选择地关注输入数据中的关键信息。以下是其核心思想的分点简述：

1. **选择性聚焦**  
   模仿人类注意力，允许模型在处理信息时忽略无关部分，集中资源于最重要的特征或上下文，提升计算效率。

2. **权重动态分配**  
   通过计算查询（Query）与键（Key）的相似度，生成注意力权重，再对值（Value）加权求和。权重随输入内容动态调整，实现灵活的信息提取。

3. **上下文依赖建模**  
   在序列任务（如翻译）中，通过关注不同位置的相关信息，建立长距离依赖关系，解决传统循环神经网络（RNN）的长程信息衰减问题。

4. **并行化与可扩展性**  
   自注意力机制（如Transformer）可并行计算所有位置的关联，避免序列顺序依赖，显著提升训练速度，并支持处理更长的输入序列。

5. **多维度信息融合**  
   多头注意力机制允许模型同时关注不同子空间的特征（如语义、语法），增强信息捕获的多样性和表达能力。

这些思想使注意力机制成为现代深度学习模型（如Transformer、BERT、GPT）的核心组件，广泛应用于自然语言处理、计算机视觉等领域。

In [7]:
# 函数调用模式，结构化任务执行能力
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        { "role": "user", "content": "请帮我查询北京的天气" }
    ],
    functions=[
        {
            "name": "get_weather",
            "description": "查询天气信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "城市名称",
                    }
                },
                "required": ["city"],
            }
        }
    ],
    function_call="auto",
)
# 函数调用结构
print(response.choices[0].message.function_call)
print(response.choices[0].message.content)

None
我目前无法直接查询实时天气数据。不过，您可以通过以下方式快速获取北京的天气信息：

1. **推荐使用**：
   - 打开手机天气App（如系统自带天气、墨迹天气等）
   - 在搜索引擎（百度/谷歌）搜索“北京天气”
   - 通过微信小程序搜索“天气预报”

2. **备用方案**：
   - 访问中国天气网（www.weather.com.cn）
   - 查看手机通知栏的天气插件

如果您需要了解天气相关的出行建议（如穿衣指南、是否需带伞等），可以告诉我您的具体需求，我会根据一般性知识为您提供参考建议。🌤️
